**Team Members: Ethan Wong, Timmy Ren, Mason Shu, Medha Nalamada, Carson Mullen, Bethel Kim**

**Morning Cohort (11 AM - 1 PM)**

*Note to all: Please pull any changes from the repo before working on this file!*

In [9]:
# Install Libraries
#!pip install deep-translator
#!pip install langdetect
#!pip install vaderSentiment

In [8]:
# Import Libraries
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    TimeoutException,
    NoSuchElementException,
    ElementClickInterceptedException,
    WebDriverException,
)
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import numpy as np
import json
from deep_translator import GoogleTranslator
from langdetect import detect
from collections import Counter
import re
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer #type: ignore

# Task A: Scrape from ratebeer.com - extract 5-6K reviews

In [ ]:
def handle_cookie_consent():
    try:
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.ID, "onetrust-banner-sdk"))
        )
        accept_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@id='onetrust-accept-btn-handler']"))
        )
        accept_button.click()
        WebDriverWait(driver, 5).until(
            EC.invisibility_of_element_located((By.ID, "onetrust-banner-sdk"))
        )
    except (TimeoutException, NoSuchElementException, WebDriverException):
        pass

def handle_ratebeer_banner():
    try:
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'RateBeerBestBanner___StyledDiv-jAauHc'))
        )
        close_button = driver.find_element(By.CSS_SELECTOR, '.RateBeerBestBanner___StyledDiv-jAauHc .MuiIconButton-root')
        close_button.click()
        WebDriverWait(driver, 5).until(
            EC.invisibility_of_element_located((By.CLASS_NAME, 'RateBeerBestBanner___StyledDiv-jAauHc'))
        )
    except (TimeoutException, NoSuchElementException, WebDriverException):
        pass

json_file_path = 'beer_reviews.json'
with open(json_file_path, 'w', encoding='utf-8') as f:
    f.write('[')

chrome_options = Options()
chrome_options.add_argument("--headless")  # Run in headless mode
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--window-size=1920,1080")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)

driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()), options=chrome_options
)
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

# Navigate to the RateBeer Top Beers page
driver.get("https://www.ratebeer.com/top-beers")

handle_cookie_consent()

WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located(
        (By.XPATH, "//div[contains(@class,'DataTable__Row')]/div[2]/a")
    )
)

# Get the list of beers and their links
beer_elements = driver.find_elements(By.XPATH, "//div[contains(@class,'DataTable__Row')]/div[2]/a")
beers = []
for beer_element in beer_elements:
    full_text = beer_element.text
    product_name = full_text.split('\n')[0]  # Take the first line before the newline
    product_link = beer_element.get_attribute('href')
    beers.append({'name': product_name, 'link': product_link})

total_reviews_collected = 0
max_reviews = 30000  # Set your desired number of reviews to collect
first_review = True  # Flag to handle commas in JSON

# Loop through all beers
for i in range(len(beers)):
    product_name = beers[i]['name']
    product_link = beers[i]['link']

    # Navigate to the beer page
    driver.get(product_link)

    handle_cookie_consent()
    handle_ratebeer_banner()

    # Wait for the dropdown to be present
    try:
        dropdown = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "MuiSelect-selectMenu"))
        )
        dropdown.click()

        # Wait for the dropdown options to be visible
        option_100 = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.XPATH, "//li[contains(text(), '100')]"))
        )
        option_100.click()
    except (TimeoutException, NoSuchElementException, WebDriverException):
        continue

    # Get the total number of reviews
    try:
        num_items_text = driver.find_element(By.CSS_SELECTOR, ".MuiTablePagination-selectRoot + .MuiTypography-colorInherit").text
        total_reviews = int(num_items_text.split(" ")[-1])
        num_pages = int(np.ceil(total_reviews / 100))
    except (NoSuchElementException, ValueError, WebDriverException):
        continue

    for j in range(num_pages):
        time.sleep(2)
        # Expand all "Show more" buttons
        while True:
            try:
                show_more_button = driver.find_element(By.XPATH, "//span[text()='Show more']")
                driver.execute_script("arguments[0].scrollIntoView(true);", show_more_button)
                show_more_button.click()
                time.sleep(1)
            except NoSuchElementException:
                break
            except ElementClickInterceptedException:
                handle_ratebeer_banner()
                driver.execute_script("arguments[0].click();", show_more_button)
            except Exception as e:
                print(f"Could not click 'Show more' button: {e}")
                break

        # Get all comments and store as a list
        comments_elements = driver.find_elements(By.CSS_SELECTOR, ".pre-wrap.MuiTypography-body1")
        product_reviews = [comment.text for comment in comments_elements]

        rating_elements = driver.find_elements(By.CSS_SELECTOR, ".bRPQdN.MuiTypography-subtitle1")
        ratings = [rating.text for rating in rating_elements]

        beer_data_list = []

        for review, rating in zip(product_reviews, ratings):
            beer_data = {
                'product_name': product_name,
                'product_review': review,
                'rating': rating
            }
            beer_data_list.append(beer_data)
            total_reviews_collected += 1

            if total_reviews_collected >= max_reviews:
                break

        # Write data to the JSON file
        with open(json_file_path, 'a', encoding='utf-8') as f:
            for beer_data in beer_data_list:
                if not first_review:
                    f.write(',\n')
                json.dump(beer_data, f, ensure_ascii=False)
                first_review = False
        beer_data_list = []

        if total_reviews_collected >= max_reviews:
            break

        if j + 1 < num_pages:
            try:
                button_to_click = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, ".MuiIconButton-colorInherit + .MuiIconButton-colorInherit"))
                )
                handle_ratebeer_banner()
                button_to_click.click()
                time.sleep(1)
            except ElementClickInterceptedException:
                handle_ratebeer_banner()
                driver.execute_script("arguments[0].click();", button_to_click)
            except Exception as e:
                print(f"Could not click 'Next' button: {e}")
                break

    if total_reviews_collected >= max_reviews:
        break

# Close the JSON array properly
with open(json_file_path, 'a', encoding='utf-8') as f:
    f.write(']')

driver.quit()

In [23]:
# Load the JSON data into a DataFrame
with open('beer_reviews.json', 'r', encoding='utf-8') as file:
    beer_reviews = json.load(file)

df = pd.DataFrame(beer_reviews)
df

,product_name,product_review,rating
0,Toppling Goliath Kentucky Brunch,"You need personal informations from companies,...",3.3
1,Toppling Goliath Kentucky Brunch,"Bottle after MBCC 2024. Black colour, malty ar...",4.5
2,Toppling Goliath Kentucky Brunch,Thank you for sharing this Chris - Black with ...,4.3
3,Toppling Goliath Kentucky Brunch,"Boxed beer at home, proper glassware. Pitch bl...",4.7
4,Toppling Goliath Kentucky Brunch,"From backlog. (As 2018 Vintage) 0,3 litre Bott...",4.9
...,...,...,...
29371,Superstition Blue Berry White,"Bottle 82/100. Pours a deep, inky purple. More...",4.8
29372,Superstition Blue Berry White,Pours a deep berry hue. Aroma of rich blueberr...,4.8
29373,Superstition Blue Berry White,"If you look at my reviews, you will see how hi...",5.0
29374,Superstition Blue Berry White,Huge thank you to Dakine for sharing this with...,4.6


In [24]:
# Downsampling Reviews
freq = pd.DataFrame(df.value_counts('product_name')).reset_index()
freq.columns = ['product_name', 'count']
filtered_df = df[df['product_name'].isin(freq[freq['count'] > 200]['product_name'])]

In [25]:
sampled_df = filtered_df.groupby('product_name', group_keys=False).apply(lambda x: x.sample(min(len(x), 100)))

In [26]:
sampled_df

,product_name,product_review,rating
8667,3 Fonteinen J & J Oude Geuze Blauw,"Bottle, 75cl corked. A rather oily pour leaves...",3.6
8739,3 Fonteinen J & J Oude Geuze Blauw,Druckigt under aliasnamnet 3 Fonteinen Akkurat...,4.7
8792,3 Fonteinen J & J Oude Geuze Blauw,"Bottle @ Heeren Van Liedekercke, Denderleeuw. ...",4.2
8821,3 Fonteinen J & J Oude Geuze Blauw,Bottle shared at Cleveland tasting on 8-23-08....,4.3
8923,3 Fonteinen J & J Oude Geuze Blauw,A Mes rate: 75cl bottle dated 6/9/2003 in homa...,4.9
...,...,...,...
2338,Westvleteren 12,Bottle. Mahogany with small beige head and nic...,4.1
2178,Westvleteren 12,"Tasted better. Appearance is great, aroma alik...",4.0
4965,Westvleteren 12,Powerfull in tastes and aromas as no others. L...,4.8
3008,Westvleteren 12,"Very nice beer, although a little overrated be...",4.6


In [27]:
sampled_df.value_counts('product_name')

product_name
3 Fonteinen J & J Oude Geuze Blauw                          100
3 Fonteinen Oude Geuze 1998 (50th Anniversary)              100
Trappistes Rochefort 10                                     100
Toppling Goliath Mornin' Delight                            100
Three Floyds Dark Lord - Bourbon Vanilla Bean               100
Three Floyds Dark Lord - Bourbon Barrel Aged                100
Russian River Pliny the Younger                             100
Perennial Abraxas - Barrel-Aged                             100
Närke Kaggen Stormaktsporter                                100
La Face Cachée de la Pomme Neige Récolte d'Hiver            100
Goose Island Bourbon County Stout - Rare 2010               100
Goose Island Bourbon County Stout - Proprietor's 2013       100
Founders KBS (Kentucky Breakfast Stout)                     100
Founders CBS (Canadian Breakfast Stout)                     100
De Dolle Stille Nacht Reserva 2000                          100
Cigar City Hunahpu's Imperi

In [8]:
#sampled_df.to_csv('sampled_beer_review.csv')

In [28]:
# Function to translate text to English if it's not already in English
def translate_to_english_if_needed(text):
    try:
        # Detect the language of the text
        lang = detect(text)
        # Translate only if the language is not English
        if lang != 'en':
            translated = GoogleTranslator(source='auto', target='en').translate(text)
            return translated
        else:
            return text
    except Exception as e:
        return str(e)

# Apply the translation function to the 'product_review' column
sampled_df['product_review'] = sampled_df['product_review'].apply(translate_to_english_if_needed)

In [11]:
# Export the DataFrame to a CSV or json file to prevent re-running the code in the future
#sampled_df.to_csv('final_beer_reviews.csv', index=False)

# Task B: Perform a word frequency analysis to find the most important beer attributes and ensure that the attributes specified are likely to be mentioned together in a review

In [5]:
#Load final reviews dataframe 
df = pd.read_csv('final_beer_reviews.csv')

#Word frequency analysis of entire df 
df['product_review'] = df['product_review'].apply(str)
all_text = ' '.join(df['product_review'].dropna())
words = re.findall(r'\w+', all_text.lower())
word_counts = Counter(words)
word_freq_df = pd.DataFrame(word_counts.items(), columns=['Word', 'Frequency']).sort_values(by='Frequency', ascending=False)

#Define a common list of attributes to search through 
attributes = [
    'aggressive', 'balanced', 'crisp', 'sweet', 'diacetyl', 'estery', 'floral', 'fruity', 'malty', 
    'robust', 'light', 'hoppy', 'bitter', 'clove', 'dry', 'earthy', 'herbal', 'nutty', 'piney', 
    'spicy', 'tart', 'woody', 'creamy', 'toast', 'smooth', 'oaky',
    'peppery', 'zesty', 'citrus', 'pungent', 'sour', 'smoky', 'fragrant', 'lingering',
    'thin', 'yeasty', 'burnt', 'rich', 'mellow', 'clean', 'dank', 
    'warming',
]

attribute_freq = word_freq_df[word_freq_df['Word'].isin(attributes)]
top_attributes = attribute_freq.sort_values(by='Frequency', ascending=False)
top_25_attributes = top_attributes[:25]['Word']
top_25_attributes_list = top_25_attributes.str.lower().tolist()
print('Top 25 Attributes:\n', top_25_attributes)

Top 25 Attributes:
 227         sweet
196         light
457        smooth
212          rich
835        bitter
220        creamy
536      balanced
172           dry
177          sour
770          thin
87         fruity
282        citrus
736       warming
191          tart
913         malty
441         spicy
1656        burnt
458         woody
349        earthy
284     lingering
637       peppery
549          oaky
281        floral
2426        hoppy
994        mellow
Name: Word, dtype: object


Before we select 3 attributes, doing a lift analysis will help us see strongly co-occuring attributes that can be used as an ideal selection

In [6]:
#Performing a lift analysis to assess their associations within a message
from itertools import combinations
from collections import defaultdict

pd.set_option('display.max_rows', None)

# Top 3 attributes
input_attributes = top_25_attributes_list

def clean_text(text):
    text = re.sub(r'\W+', ' ', text.lower())  # Remove punctuation and convert to lowercase
    words = text.split()
    return words 

def find_attribute_pairs(message, attributes, window=50):
    message = re.sub(r'\W+', ' ', message.lower())  # Clean the message
    words = message.split()
    attribute_indices = {attribute: [] for attribute in attributes}

    # Track the position of each attribute in the message
    for i, word in enumerate(words):
        if word in attributes:
            attribute_indices[word].append(i)

    # Find pairs within the given window
    attribute_pairs = set()
    attribute_list = list(attribute_indices.keys())
    for idx1, attribute1 in enumerate(attribute_list):
        indices1 = attribute_indices[attribute1]
        for idx2 in range(idx1 + 1, len(attribute_list)):
            attribute2 = attribute_list[idx2]
            indices2 = attribute_indices[attribute2]
            for i1 in indices1:
                for i2 in indices2:
                    if abs(i1 - i2) - 1 <= window:
                        pair = tuple(sorted((attribute1, attribute2)))
                        attribute_pairs.add(pair)
    return attribute_pairs

# Function to calculate word frequencies and word pair co-occurrences
def calculate_frequencies(df, input_attributes, window=50):
    word_frequency = defaultdict(int)
    word_pair_frequency = defaultdict(int)

    # Loop through each post and count word occurrences
    for index, row in df.iterrows():
        message = row['product_review']
        words_in_message = set(clean_text(message))
        attributes_in_message = words_in_message.intersection(input_attributes)
        
        # Update word frequencies
        for attribute in attributes_in_message:
            word_frequency[attribute] += 1

        # Update pair frequencies
        attribute_pairs = find_attribute_pairs(message, input_attributes, window=window)
        for attribute_pair in attribute_pairs:
            if attribute_pair[0] in attributes_in_message and attribute_pair[1] in attributes_in_message:
                word_pair_frequency[attribute_pair] += 1

    return word_frequency, word_pair_frequency


# Function to calculate lift
def calculate_lift(word_frequency, word_pair_frequency, total_posts):
    lift_values = []

    for (attribute1, attribute2), pair_count in word_pair_frequency.items():
        p_attribute1 = word_frequency[attribute1] / total_posts
        p_attribute2 = word_frequency[attribute2] / total_posts
        p_both = pair_count / total_posts

        if p_attribute1 * p_attribute2 > 0:
            lift = p_both / (p_attribute1 * p_attribute2)
            lift_values.append({
                'Attribute 1': attribute1,
                'Attribute 2': attribute2,
                'Lift': lift
            })

    return pd.DataFrame(lift_values)

# Main function to process data and compute lift ratios
def main(df, top_10_brands_list, window=50):
    # Step 1: Calculate word and word pair frequencies
    total_posts = len(df)
    word_frequency, word_pair_frequency = calculate_frequencies(df, input_attributes, window)
    
    # Step 2: Calculate lift values
    lift_df = calculate_lift(word_frequency, word_pair_frequency, total_posts)
    
    # Step 3: Create lift matrix
    lift_matrix = lift_df.pivot(index='Attribute 1', columns='Attribute 2', values='Lift')
    
    # Ensure all attributes are present in both axes
    all_attributes = sorted(set(input_attributes))
    lift_matrix = lift_matrix.reindex(index=all_attributes, columns=all_attributes)
    
    # Replace missing values with 0
    lift_matrix = lift_matrix.fillna(0)
    
    # Convert DataFrame to object type to allow setting string values
    lift_matrix = lift_matrix.astype(object)
    
    # Set the diagonal and lower triangle to '-'
    for i in range(len(all_attributes)):
        lift_matrix.iloc[i, i] = '-'  # Set diagonal to '-'
        for j in range(i):
            lift_matrix.iloc[i, j] = '-'  # Set lower triangle to '-'
    
    # Remove labels from being displayed
    lift_matrix.index.name = None
    lift_matrix.columns.name = None
    
    # Return both lift_matrix and lift_df
    return lift_matrix, lift_df

# Execute the main function and capture the lift matrix and lift DataFrame
lift_matrix, lift_df = main(df, input_attributes, window=5000)

#Filter the matrix to include only high associations
filtered_lift_df = lift_df[(lift_df['Lift'] > 2) & (lift_df['Lift'] <= 4)]

In [7]:
#Function to select three attributes that have the highest lifts among all trio combinations
def find_top_trios(filtered_lift_df, top_n=1):
    
    trio_lift_sums = defaultdict(float)
    
    # Create a set of all unique attributes involved in the filtered lift pairs
    unique_attributes = set(filtered_lift_df['Attribute 1']).union(set(filtered_lift_df['Attribute 2']))
    
    # Generate all possible combinations of three attributes
    for trio in combinations(unique_attributes, 3):
        a1, a2, a3 = sorted(trio)
        
        # Get the lift values for each pair in the trio
        lift_a1_a2 = filtered_lift_df[(filtered_lift_df['Attribute 1'] == a1) & (filtered_lift_df['Attribute 2'] == a2)]['Lift']
        lift_a1_a3 = filtered_lift_df[(filtered_lift_df['Attribute 1'] == a1) & (filtered_lift_df['Attribute 2'] == a3)]['Lift']
        lift_a2_a3 = filtered_lift_df[(filtered_lift_df['Attribute 1'] == a2) & (filtered_lift_df['Attribute 2'] == a3)]['Lift']
        
        # Calculate the sum
        if not lift_a1_a2.empty and not lift_a1_a3.empty and not lift_a2_a3.empty:
            trio_lift_sums[trio] = lift_a1_a2.values[0] + lift_a1_a3.values[0] + lift_a2_a3.values[0]

    # Sort the trios
    sorted_trios = sorted(trio_lift_sums.items(), key=lambda x: x[1], reverse=True)[:top_n]
    top_trios = [{'Attributes': triad, 'Lift Sum': lift_sum} for triad, lift_sum in sorted_trios]
    
    return pd.DataFrame(top_trios)

# Display the top 3 trios
top_three_trios = find_top_trios(filtered_lift_df, top_n=3)
top_three_trios

,Attributes,Lift Sum
0,"(earthy, citrus, sour)",10.650483
1,"(peppery, hoppy, lingering)",10.245056
2,"(tart, earthy, citrus)",10.046699


Based on our finding here, we will use earthy, citrus, and sour as the three attributes chosen by the customer!

# Task C: Perform a similarity analysis using cosine similarity with the three attributes specified by the customer and the reviews

**Note: Used the bag-of-words model as opposed to word embeddings**

In [19]:
# Script should accept a file as input that has the product attributes and calculate similarity scores (0-1) 
    # between these attributes and each review
# Output file should have three columns: product_name (will have a row for each review), product_review, 
    # and similarity_score

In [9]:
# Function to calculate cosine similarity between attributes and multiple reviews 
# based on cosine similarity code Dr. Barua posted
def calculate_similarity(attributes, reviews):
    # Combine the attributes into one string (bag of words)
    attributes_combined = ' '.join(attributes)
    
    # Combine the reviews into a list (to be vectorized) and add attributes as the first document
    documents = [attributes_combined] + list(reviews)  # First document is the attributes
    
    # Use CountVectorizer to convert text to bag-of-words representation
    count_vectorizer = CountVectorizer(stop_words='english')
    sparse_matrix = count_vectorizer.fit_transform(documents)  # Vectorize all documents
    
    # Convert sparse matrix to dense (using todense()) for cosine similarity calculation
    doc_term_matrix = sparse_matrix.todense()  # Using todense() as in the reference code
    
    # Create a DataFrame from the term-document matrix
    df = pd.DataFrame(doc_term_matrix, columns=count_vectorizer.get_feature_names_out(), index=['attributes'] + [f'review_{i}' for i in range(len(reviews))])
    
    # Calculate cosine similarity between the first document (attributes) and each review
    similarity_scores = cosine_similarity(df.iloc[0:1], df.iloc[1:]).flatten()  # Skip first row (attributes)
    
    return similarity_scores
    #return attributes_combined

In [10]:
# Execute cosine similarity function and capture similarities for chosen attributes 

attributes = ['citrus', 'earthy', 'sour']  

reviews = df['product_review']
product_names = df['product_name'] 

# Calculate similarity for all reviews
similarity_scores = calculate_similarity(attributes, reviews)

# Create a new DataFrame with product_name, product_review, and similarity_score
output_df = pd.DataFrame({
    'product_name': product_names,
    'product_review': reviews,
    'similarity_score': similarity_scores
})

In [11]:
# Sort output_df by similarity_score in descending order (highest to lowest)
output_df_sorted = output_df.sort_values(by='similarity_score', ascending=False)

# Display the first few rows of the sorted DataFrame
output_df_sorted.head()

,product_name,product_review,similarity_score
64,3 Fonteinen J & J Oude Geuze Blauw,(Bottle 75 cl) Courtesy of yespr. Pours a clou...,0.400320
156,3 Fonteinen Oude Geuze 1998 (50th Anniversary),"Beer #10000! Bottle at Akkurat, Stockholm. Cle...",0.383598
814,De Dolle Stille Nacht Reserva 2000,"#1900, thanks to bu11zeye, cloudy color of gra...",0.361158
107,3 Fonteinen Oude Geuze 1998 (50th Anniversary),"Bottled @RBESG’05\n\nPale yellow, creamy white...",0.353553
126,3 Fonteinen Oude Geuze 1998 (50th Anniversary),"Big, big thanks Jason for opening this one up ...",0.348155


In [12]:
# Count rows where similarity_score is greater than 0
rows_with_similarity = output_df[output_df['similarity_score'] > 0].shape[0]
print(f"Number of rows with similarity score > 0: {rows_with_similarity}")

Number of rows with similarity score > 0: 277


In [98]:
#output_df.to_csv('cos_similiarities.csv', index=False)

# Task D: Perform a sentiment analysis for every review

In [13]:
# Viewing sentiment scores for each word in the reviews to determine if alterations are needed

analyzer = SentimentIntensityAnalyzer()

all_reviews = ' '.join(df['product_review'].tolist())
words = set(all_reviews.split())

word_scores = {}

for word in words:
    if word in analyzer.lexicon:
        word_scores[word] = analyzer.lexicon[word]
    else:
        word_scores[word] = 'Not in VADER lexicon'

word_scores_df = pd.DataFrame(list(word_scores.items()), columns=['Word', 'Sentiment Score'])

In [14]:
# Print out the sentiment scores for each word
filtered_df = word_scores_df[word_scores_df['Sentiment Score'] != 'Not in VADER lexicon']

for index, row in filtered_df.iterrows():
    print(f"Word: {row['Word']}, Sentiment Score: {row['Sentiment Score']}")

Word: stout, Sentiment Score: 0.7
Word: inviting, Sentiment Score: 1.3
Word: negative, Sentiment Score: -2.7
Word: unbelievable, Sentiment Score: 0.8
Word: dull, Sentiment Score: -1.7
Word: huge, Sentiment Score: 1.3
Word: loved, Sentiment Score: 2.9
Word: leave, Sentiment Score: -0.2
Word: bittersweet, Sentiment Score: -0.3
Word: bullied, Sentiment Score: -3.1
Word: frustrated, Sentiment Score: -2.4
Word: hopes, Sentiment Score: 1.8
Word: excuse, Sentiment Score: 0.3
Word: treat, Sentiment Score: 1.7
Word: cutting, Sentiment Score: -0.5
Word: plays, Sentiment Score: 1.0
Word: allow, Sentiment Score: 0.9
Word: forget, Sentiment Score: -0.9
Word: powerful, Sentiment Score: 1.8
Word: likes, Sentiment Score: 1.8
Word: harmony, Sentiment Score: 1.7
Word: worse, Sentiment Score: -2.1
Word: creates, Sentiment Score: 1.1
Word: gracious, Sentiment Score: 2.6
Word: wth, Sentiment Score: -2.4
Word: alright, Sentiment Score: 1.0
Word: attacks, Sentiment Score: -1.9
Word: tough, Sentiment Score: -

Based on the these scores, it might be best to adjust terms related to bitterness, light, complex and add terms like sour and aggressive!

In [15]:
new_words = {
    'bitter': 0.5,  # Adjusted from negative to slightly positive for IPAs
    'bitterness': 0.5,
    'light': 1.0,  # Adjusted to reflect a more neutral-positive trait
    'low': 0.5,  # Adjusted to neutral for certain beer styles
    'strong': 2.5,  # Slight increase for beer context
    'sour': 0.8,  # Positive adjustment for sour beer styles
    'complex': 2.0,  # Higher for describing flavor complexity
    'aggressive': 0.5,  # Neutralized in the context of bold beer flavors
}
analyzer.lexicon.update(new_words)

In [16]:
#Calculating sentiment for each review

analyzer = SentimentIntensityAnalyzer()

def get_sentiment_vader(review):
    sentiment_scores = analyzer.polarity_scores(review)
    return sentiment_scores['compound']  

# Apply the VADER sentiment analysis to each product review
df['sentiment_score'] = df['product_review'].apply(get_sentiment_vader)

In [17]:
df[df['sentiment_score'] < 0]

,product_name,product_review,rating,sentiment_score
13,3 Fonteinen J & J Oude Geuze Blauw,Bottle. Cloudy golden colour with a little hea...,3.1,-0.0516
28,3 Fonteinen J & J Oude Geuze Blauw,"Bottle thanks to mes, i havnt rated in a week ...",4.9,-0.2890
41,3 Fonteinen J & J Oude Geuze Blauw,"Aroma/Malty-light,\nAppearance Head/Initial-sm...",1.9,-0.3412
67,3 Fonteinen J & J Oude Geuze Blauw,Bottle at Max & Jess’s for an impromptu barbec...,3.9,-0.4404
85,3 Fonteinen J & J Oude Geuze Blauw,Way worse than druiven. I also don’t like the ...,1.3,-0.1531
111,3 Fonteinen Oude Geuze 1998 (50th Anniversary),"Bottle. Very tart, sour and dry, farmyard, hay...",3.1,-0.4767
113,3 Fonteinen Oude Geuze 1998 (50th Anniversary),"30th December 2014\n5 Kay, 7.5 Kay, 10 Kay Tas...",3.2,-0.8836
115,3 Fonteinen Oude Geuze 1998 (50th Anniversary),"Lemon oak, with a musty finish. Drank along si...",3.6,-0.5423
127,3 Fonteinen Oude Geuze 1998 (50th Anniversary),"The Beer God brought this, and he’s just cool....",4.3,-0.1069
141,3 Fonteinen Oude Geuze 1998 (50th Anniversary),Amber with little white head. Aroma is cherry ...,4.0,-0.2075


# Task E: Create an evaluation score for each beer that uses both similarity and sentiment scores and recommended three products to the customer

In [18]:
# Add sentiment and similarity scores for the three products recommended.
# Create a df with product_name, product_review, similarity, sentiment, evaluation
# Use all similarity rows
# Recommend top 3 products based on evaluation

df_combined = pd.merge(df, output_df[['product_name', 'product_review', 'similarity_score']], 
                       on=['product_name', 'product_review'], how='inner')

# Calculate evaluation score
df_combined['evaluation_score'] = df_combined['similarity_score'] * df_combined['sentiment_score']

# Group by product_name and calculate the mean evaluation score for each beer
product_evaluation = df_combined.groupby('product_name')['evaluation_score'].mean().reset_index()

# Rank products
top_products = product_evaluation.sort_values(by='evaluation_score', ascending=False)

# Recommend top 3
top_3_products = top_products.head(3)
print(top_3_products)

                                      product_name  evaluation_score
0               3 Fonteinen J & J Oude Geuze Blauw          0.077530
1   3 Fonteinen Oude Geuze 1998 (50th Anniversary)          0.071638
16                 Russian River Pliny the Younger          0.035876


# Task F: How would our recommendations change if we use word vectors instead of the standard bag-of-words cosine similarity?

In [22]:
# Consider the % of reviews that mention a preferred attribute
    # For a recommended product, what % of its reviews mention an attribute specified by the customer
# Differences between bag-of-words and word vector approaches
# Bag of words, under certain conditions, will likely be better than word embeddings for recommender systems
# Show rating, similarity scores, and sentiments for the products recommended in this task and the previous one

# Task G: How would our recommendations differ if we ignored the similarity and feature sentiment scores and just chose the 3 highest rated products fro the entire data set? Would these products meet the requirements of the user looking for recommendations? Why or why not?

In [23]:
# Justify with analysis - use similarity and sentiment scores as well as overall ratings to answer this questions

# Task H: From 10 beers in the data, choose 1, and find the most similar beer among the remaining 9

In [24]:
# Show logic we are using in addition to finding the most similar product with markdown; explain method/logic